# 6.2 Region-content description

Unlike shape description techniques, which work with regions' contours, region-content description focuses on characterizing the content of segmented regions through their distribution in the image, their textures, etc. Regarding textures, it gives us information about the spatial arrangement of color or intensities in an image or selected region of an image. Textures can be used to help in segmentation or classification of images. Notice that these methods doesn't require binary images as input. 

This notebook covers different region-content description techniques:

- 2D image moments (<a href="#621">section 6.2.1</a>)
- Hu moments (<a href="#622">section 6.2.2</a>)
- Image histogram moments (<a href="#6231">section 6.2.3.1</a>)
- Co-ocurrence matrices (<a href="#6232">section 6.2.3.2</a>)


## Problem context - Car plates

In this notebook, our task is twofold!

### Number-plate detection for UMA

<center><img src="./images/access_system.png" width="400"></center>$\\[5pt]$

Basically, we have to continue with our number-plate detection work looking for a way to obtain a feature vector that distinguishes each character in a Spanish car plate. In this notebok we will try more advanced methods, like **image moments** or **Hu moments**.

### Identification of the State of a license plate state

An American company contacted us for developing a **texture description method** that describes **a license plate according to its State of origin** instead of the characters appearing. As you may know, USA uses a different license plates for each state in the country:$\\[5pt]$

<center><img src="./images/usa_plates.jpg" width="600"></center>

You will use some region description methods applied to this problem like **co-ocurrence matrices** or **image histogram moments**. Again, your task is to develop a method returning a feature vector that allows for the identification of the State of origin of such license plates.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (15.0, 8.0)
from scipy import stats 

images_path = './images/'

## 6.2.1 Image moments <a id="621"></a>

An **image moment** (2D-moment) is the weighted average (or moment) of the intensity of the pixels in the image/region, or a function combining other moments. Moments usually have some attractive property or interpretation, and they can work in both grayscale and color images. For example, when working with 1 dimension (*e.g.* with a histogram), the moment of order 0 represents the number of pixels in the image, while when dealing with 2 dimensions (*e.g.* an image) it represents its area, that is, the number of white pixels (if the image is binary).

There are 3 main types of moments:

- **Non-central moments:**

  $\hspace{1.2cm}m_{ij} = \sum_{y=1}^{rows}\sum_{x=1}^{cols}x^i y^j I(y,x)$
  $\\[5pt]$
   $\hspace{1.2cm}$where $I(y,x)$ represents the intensity of the pixel in the $(y, x)$ coordinates of image $I$.$\\[5pt]$

- **Central moments:**

  $\hspace{1.2cm}\mu_{ij} = \sum_{y=1}^{rows}\sum_{x=1}^{cols}(x-\overline{x})^i (y-\overline{y})^j I(y,x)$ 
  $\\[5pt]$
   $\hspace{1.2cm}$being $\left(\overline{x},\overline{y}\right) = \left(\frac{m_{10}}{m_{00}} ,\frac{m_{01}}{m_{00}} \right)$ the centroid of the region. $\\[3pt]$
 
  When dealing with big images/regions, it is possible to save some computation time computing the central moments using the non-central ones:

$
\begin{array}{l}
\hspace{2cm}\mu_{00} = m_{00} \equiv \mu \\
\hspace{2cm}\mu_{01} = 0 \\
\hspace{2cm}\mu_{10} = 0 \\
\hspace{2cm}\mu_{20} = m_{20} - \mu \overline{x}^2\\
\hspace{2cm}\mu_{11} = m_{11} - \mu \overline{x}\,\overline{y}\\
\hspace{2cm}\mu_{02} = m_{02} - \mu \overline{y}^2\\
\hspace{2cm}\mu_{30} = m_{30} - 3m_{20} \overline{x} + 2 \mu \overline{x}^3 \\
\hspace{2cm}\mu_{21} = m_{21} - m_{20} \overline{y} - 2m_{11} \overline{x} + 2\mu \overline{x}^2\overline{y} \\
\hspace{2cm}\mu_{12} = m_{12} - m_{02} \overline{x} - 2m_{11} \overline{y} + 2\mu \overline{y}^2\overline{x}\\
\hspace{2cm}\mu_{03} = m_{03} - 3m_{02} \overline{y} + 2 \mu \overline{y}^3\\
\end{array}
$<br /> <br />
$\hspace{1cm}$In general, the following formula can be used to retrieve an arbitrary central moment: <br /><br />
$\hspace{2cm}\mu_{pq} = \sum_m^p \sum_n^q {p\choose m}{q\choose n}(-\overline{x})^{(p-m)}(-\overline{y})^{(q-n)}m_{mn}$$\\[5pt]$

- **Scale invariant moments:**$\\[3pt]$<br />
Can be built from from central moments by dividing through a properly scaled zero-th central moment:

$\hspace{2cm} \eta_{ij} = \mu_{ij}\ /\ \mu_{00}^{1+((i+j)/2)}$

$\hspace{2cm}$ where $i + j \ge 2$.

#### <font color="orange">OpenCV pill</font>
    
OpenCV defines a method for computing some central, non-central and scale-invariant moments called [`cv2.moments()`](https://docs.opencv.org/4.2.0/d3/dc0/group__imgproc__shape.html#ga556a180f43cab22649c23ada36a8a139), which gets:

- working with intensity images: a contour (array of 2D points) delimiting the segmented regions.
- working with grayscale images: the image itself.

This function returns a dictionary containing the computed moments.

### **<span style="color:green"><b><i>ASSIGNMENT 1: Computing image moments</i></b></span>**

**What to do?** Your first tasks is to complete the method `image_moments()`, which applies the previously mentioned [`cv2.moments()`](https://docs.opencv.org/4.2.0/d3/dc0/group__imgproc__shape.html#ga556a180f43cab22649c23ada36a8a139) to a binary image, for example a thresholded image containing the numbers of a plate. 

In [ ]:
# Assignment 1
def image_moments(region):
    """ Compute moments of the region in a binary image.   
    
        Args:
            region: Binary image
                    
        Returns: 
            moments: dictionary containing all moments of the region
    """   
    # Compute moments
    moments = cv2.moments(None)
    
    return moments

You can use the next code to **test if the results are correct**, rounding the output of your `image_moments()` function to have 2 decimals:

In [ ]:
region = np.array([[255,255,255,255,255],[255,0,0,0,255],[255,0,0,255,255],[255,0,255,255,255],[0,0,255,255,255]], dtype=np.uint8)
moments = image_moments(region)

# Round moments for visualization matters
for k, v in moments.items():
    moments[None] = round(None,None)

print(moments)

<font color='blue'>**Expected output  </font>**

    {'m00': 4335.0, 'm10': 9945.0, 'm01': 8160.0, 'm20': 32895.0, 'm11': 20655.0, 'm02': 24990.0, 'm30': 115515.0, 'm21': 68595.0, 'm12': 65535.0, 'm03': 83130.0, 'mu20': 10080.0, 'mu11': 1935.0, 'mu02': 9630.0, 'mu30': -6199.41, 'mu21': -2203.24, 'mu12': 920.29, 'mu03': -164.12, 'nu20': 0.0, 'nu11': 0.0, 'nu02': 0.0, 'nu30': -0.0, 'nu21': -0.0, 'nu12': 0.0, 'nu03': -0.0}

The next code illustrates the moments retrieved from two toy images containing a square, a corner and a line. As you can see, they show differences, so these moments postulate as good descriptors for differentiating them. 

Recall some interesting facts: 

- Number of white pixels in the image (if binary): $m_{00} = \sum_{y=1}^{rows}\sum_{x=1}^{cols}I(y,x)$
- Centroid: $(\overline{x},\overline{y})=(\frac{m_{10}}{m_{00}}, \frac{m_{01}}{m_{00}})$
- Excentricity: $e=\frac{(\mu_{20}-\mu_{02})^2+4\mu_{11}^2}{(\mu_{20}+\mu_{02})^2}$ *(Ratio of the longest chord and longest perpendicular chord)*

In [ ]:
def print_features(moments):
    print('\n'+'Some features:')
    # Excentricity
    e = ((moments['mu20']-moments['mu02'])**2+4*(moments['mu11']**2))/((moments['mu20']+moments['mu02'])**2)
    print('Number of white pixels =',moments['m00']/255)
    print('Centroid (x,y) = (',moments['m10']/moments['m00'],',',
     moments['m01']/moments['m00'],')')
    print('Excentricity e = ', round(e,2), '\n')
    
    
im_square = np.array([[0,0,0,0],[0,255,255,255],[0,255,255,255],[0,255,255,255]], dtype=np.uint8)
im_corner = np.array([[0,255,255,255],[0,255,0,0],[0,255,0,0],[0,255,0,0]], dtype=np.uint8)
im_line = np.array([[0,255,0,0],[0,255,0,0],[0,255,0,0],[0,255,0,0]], dtype=np.uint8)

moments_square = image_moments(im_square)
moments_corner = image_moments(im_corner)
moments_line = image_moments(im_line)

for k, v in moments_square.items():
    moments_square[k] = round(v,2)
    
for k, v in moments_corner.items():
    moments_corner[k] = round(v,2)
    
for k, v in moments_line.items():
    moments_line[k] = round(v,2)    

plt.subplot(131)
plt.imshow(im_square,cmap='gray')
plt.title('Square image')
print('Moments square image: ' + str(moments_square))
print_features(moments_square)

plt.subplot(132)
plt.imshow(im_corner,cmap='gray')
plt.title('Corner image')
print('Moments corner image: ' + str(moments_corner))
print_features(moments_corner)

plt.subplot(133)
plt.imshow(im_line,cmap='gray')
plt.title('Line image')
print('Moments line image:   ' + str(moments_line))
print_features(moments_line)



### Invariance analysis 

Image moments could be good descriptors for addressing the problem posed by UMA. We could compute the moments of a segment region and use them as feature vector $\mathbf{x}=[x_1,\dots,x_n]^T$. Also, as $i$ and $j$ in the previous equations can take any integer, we could have a feature vector of any desired length. For example, if you design a region descriptor system that considers the first three non-central, central, and scale invariant moments, you will use: $\mathbf{x}=[m_{00},m_{10},m_{01},\mu_{20},\mu_{11},\mu_{02},\eta_{20},\eta_{11},\eta_{02}]^T$.

In the context of the number-plate detection problem, the results have to be (at least) position and scale invariants, because a car could stop closer or further away from the camera and in different positions (rotation is not that important).

To check if these moments have such invariances, you are going to compute the moments of a region, as well as of a scaled and rotated versions of it. To visually check the results, we are going to use bar charts, showing the moments for the original, rotated and scaled images, which should look like this:

<center><img src="./images/moments.png" width="800"></center>

### **<span style="color:green"><b><i>ASSIGNMENT 2: Checking invariances</i></b></span>**

Complete the method `compare_moments()`, which takes:
- a list of labels for the bar chart, and 
- three lists containing the moments of a region, and its rotated and scaled versions. 
These methods, using said arguments, plots the chart bar previously showed.

For the plot you can use `plt.bar(labels,values)`, where `labels` is a list of strings (e.g. `keys` of the dictionary of moments) and `values` a list of numbers (e.g. `values` of such dictionary). 

In [ ]:
# Assignment 2
def compare_moments(labels, moments, moments_rotated, moments_scaled):
    """ Plot a bar chart comparing the three input moment arrays  
    
        Args:
            labels: Labels of the bar chart
            moments: list containing moments of a original region
            moments_rotated: list containing moments of the original region, but previously rotated
            moments_scaled: list containing moments of the original region, but previously scaled
    """ 
    
    # Show original moments
    plt.subplot(131)
    plt.title("Original")
    plt.bar(None,None)

    # Show rotated moments
    plt.subplot(132)
    plt.title("Rotated")
    plt.bar(None,None)

    # Show scaled moments
    plt.subplot(133)
    plt.title("Scaled")
    plt.bar(None,None) 

We are going to separately analyze the invariance of:
- the **non-central** (first 10 values of moment dictionary), 
- **central** (following 7), and 
- **scale-invariant** (last 7) moments. 

*Take a look at the result of the previous assignment to check this!*

First, let's compute the moments from an initial image, a rotated version of it (90 degrees), and a scaled version (by a factor of 2 in both horizontal and vertical axes). Finally show those images.

*Hint: You can rotate a numpy array using [`np.rot90()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.rot90.html) and scale an image using [`cv2.resize()`](https://docs.opencv.org/4.2.0/da/d54/group__imgproc__transform.html#ga47a974309e9102f5f08231edc7e7529d), although there are many more options.*

In [ ]:
# Read binary image and compute moments
region = cv2.imread(images_path + 'region_6.png',0)
moments = image_moments(None)

# Rotate image and compute moments
region_rotated = np.rot90(None)
moments_rotated = image_moments(None)

# Resize image and compute moments
region_scaled = cv2.resize(None, dsize=None, fx=None, fy=None) # keep the dsize=None
moments_scaled = image_moments(None)

# Show the initial image
plt.subplot(131)
plt.title('Initial image')
plt.imshow(None, None')

# Show the rotated version
plt.subplot(132)
plt.title('Rotated version')
plt.imshow(None, None)

# Show the scaled version
plt.subplot(133)
plt.title('Scaled version')
plt.imshow(None, None);

Now, let's start comparing the **Non-central moments** of the three images! *Hint: pay special attention to the scale of the axes in the plot!*

In [ ]:
# Compare results for non-central moments
labels = list(moments.keys())[:None]
non_central_moments = list(moments.values())[:None]
non_central_rotated = list(moments_rotated.values())[:None]
non_central_scaled = list(moments_scaled.values())[:None]

compare_moments(None,None,None,None)

### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

Now, **answer the following questions:**

- Are these moments invariant to rotation?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Are these moments invariant to scale?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   

Let's continue with **central moments**!

In [ ]:
# Compare results for central moments

labels = list(moments.keys())[None:None]
central_moments = list(moments.values())[None:None]
central_rotated = list(moments_rotated.values())[None:None]
central_scaled = list(moments_scaled.values())[None:None]

compare_moments(None,None,None,None)

### <font color="blue"><b><i>Thinking about it (2)</i></b></font>

Now, **answer the following questions:**

- Are these moments invariant to rotation?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Are these moments invariant to scale?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  

And we conclude with **scale-invariant moments**.

In [ ]:
# Compare results for scale-invariant moments

labels = list(moments.keys())[None:]
invariant_moments = list(moments.values())[None:]
invariant_rotated = list(moments_rotated.values())[None:]
invariant_scaled = list(moments_scaled.values())[None:]

compare_moments(None,None,None,None)

### <font color="blue"><b><i>Thinking about it (3)</i></b></font>

Now, **answer the following questions:**

- Are these moments invariant to rotation?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Are these moments invariant to scale?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  

## 6.2.2 Hu moments <a id="622"></a>

The **Hu moments** (published in 1962 by Ming-Kuei Hu) are a set of 7 particular moments showing **interesting invariance properties**. They are calculated using scale-invariant ones:

$
\begin{array}{l}
\hspace{2cm}v_{1} = \eta_{20} + \eta_{02} \\
\hspace{2cm}v_{2} = (\eta_{20} - \eta_{02}) + 4\eta_{11}^2 \\
\hspace{2cm}v_{3} = (\eta_{20} - 3\eta_{12})^2 + (3\eta_{21} - \eta_{03})^2 \\
\hspace{2cm}v_{4} = (\eta_{30} + \eta_{12})^2 + (\eta_{21} + \eta_{03})^2\\
\hspace{2cm}v_{5} = (\eta_{30}-3\eta_{12})(\eta_{30}+\eta_{12})[(\eta_{30}+\eta_{12})^2 -3(\eta_{21}+\eta_{03})^2] + (3\eta_{21}-\eta_{03})(\eta_{21}+\eta_{03})[3(\eta_{30}+\eta_{12})^2-(\eta_{21} + \eta_{03})^2]\\
\hspace{2cm}v_{6} = (\eta_{20}-\eta_{02})[(\eta_{30}+\eta_{12})^2 -(\eta_{21}-\eta_{03})^2 + 4\eta_{11} (\eta_{30}+\eta_{12})(\eta_{21}+\eta_{03})] \\
\hspace{2cm}v_{7} = (3\eta_{21}-\eta_{03})(\eta_{30}+\eta_{12})[(\eta_{30}+\eta_{12})^2-3(\eta_{30}+\eta_{12})^2]+(\eta_{30}-3\eta_{12})(\eta_{21}+\eta_{03})[3(\eta_{30}+\eta_{12})^2-(\eta_{21}+\eta_{03})^2]\\
\end{array}
$

being:
$ 
\eta_{pq} = \frac{\mu_{pq}}{\mu^\gamma_{00}} \, \, \, \, \,  \gamma = \frac{p+q}{2}+1
$

#### <font color="orange">OpenCV pill</font>

OpenCV provides a method to retrieve the Hu moments, called (wait for it...) [`cv2.HuMoments()`](https://docs.opencv.org/4.2.0/d3/dc0/group__imgproc__shape.html#gab001db45c1f1af6cbdbe64df04c4e944)!. This method takes as input the dictionary of moments returned by `cv2.moments`. Recall that the scale-invariant moments used for their computation are the `nuij` moments in the dictionary.

### **<span style="color:green"><b><i>ASSIGNMENT 3: Exploring Hu moments invariances</i></b></span>**

Previously, we tested the invariances of non-central, central and scale-invariant moments. Now, **we are interested in checking the invariances of the Hu moments**, so we can verify if they are more suitable for the UMA parking problem. 

For that, use your brand-new `compare_moments()` function in the same way as in the previous exercises.

In [ ]:
# Assignment 3

# Read binary image and compute Hu moments
region = cv2.imread(images_path + 'region_J.png',0)
moments = image_moments(None)
hu = cv2.HuMoments(None)

# Rotate image and compute Hu moments
region_rotated = np.rot90(None)
moments_rotated = image_moments(None)
hu_rotated = cv2.HuMoments(None)

# Resize image and compute Hu moments
region_scaled = cv2.resize(None, dsize=None, fx=None, fy=None) # keep the dsize=None
moments_scaled = image_moments(None)
hu_scaled = cv2.HuMoments(None)


# Compare results for Hu moments
labels = ["hu1","hu2","hu3","hu4","hu5","hu6","hu7"]
hu_moments = hu.flatten()
hu_rotated = hu_rotated.flatten()
hu_scaled = hu_scaled.flatten()

compare_moments(None,None,None,None)

print('hu_moments:' + str(hu_moments))
print('hu_rotated:' + str(hu_rotated))
print('hu_scaled: ' + str(hu_scaled))

### <font color="blue"><b><i>Thinking about it (4)</i></b></font>

Now, **answer the following questions:**

- Are these moments invariant to rotation?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Are these moments invariant to scale?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  
    
- Now that you can deal with different ways to describe a binary region, **what descriptor would you use** for the UMA parking problem? **Why?**

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p> 

## 6.2.3 Texture 

The previous techniques are useful for describing the distribution of the regions over the image. There is another brunch of algorithms that pursuit the description of regions by **characterizing the texture of the pixels they enclose**. Such methods measure the spatial arrangement of the colors/intensities in a region, providing information about their smoothness, coarseness, and regularity. In this way, if a region does not present changes in intensity, we say that it is a untextured region.

<center><img src="./images/examples_of_different_textures.png"  width="700"/></center>

Usually, texture descriptors have spatial (position, orientation and scale) and radiometric (contrast and brightness) invariance. We are going to explore two of these descriptors:

- 1D moments of the histogram, and
- Gray Level Co-Occurrence Matrix (GLCM)

### 6.2.3.1 1D moments of histogram <a id="6231"></a>

The **central moments of the histogram** of the pixels within a region statistically describes the frequency of their intensities. They permit us to compactly describe the region through a feature vector containing a few features. They are computed using the equation: $\\[5pt]$

$\hspace{2cm} \mu_n = \sum_{i=0}^{255} (z_i - \overline{z})^n h(z_i)$

where $h(z_i)$ represents the value stored in the histogram $h(\cdot)$ for the intensity $z_i$. Keep in mind that:

$
\begin{array}{l}
\hspace{2cm} \mu_0: \text{number of pixels (1 if normalized)} \\
\hspace{2cm} \mu_1 = 0 \\ 
\hspace{2cm} \mu_2: \text{variance (contrast)} \\
\hspace{2cm} \mu_3: \text{histogram skew} \\
\hspace{2cm} \mu_4: \text{histogram uniformity} \\
\end{array}
$

However, they have a serious drawback: they don't encode pattern structures, so different textures may have similar histograms:

<center><img src="./images/patterns.png"/></center>

Nevertheless, they can be a good option depending on the application, so do not underestimate them!

### **<span style="color:green"><b><i>ASSIGNMENT 4: Analyizing histograms</i></b></span>**

In order to play a bit with these moments, we move to our second application: the state recognition in USA car plates. Two examples of such license plates: $\\[12pt]$

<center>
    <img src="./images/idaho.jpg" width="300" />
    <img src="./images/nevada.jpg" width="300" /> 
    <figcaption>Two examples of USA car plates</figcaption>
</center>

As we can see, the main difference between them is the texture in the plate background, as each state has a different one. Let's try 1D moments of the histogram for describing those textures!

**Your first task** is to plot the histogram of the previous images: `nevada.jpg` and `hawaii.jpg`, and check if the shape of the histograms is enough to differentiate them. *Hint: recall the [`np.ravel()`](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html) function*

In [ ]:
# Assignment 4

# Read images
nevada = cv2.imread(images_path + 'nevada.jpg',0)
hawaii = cv2.imread(images_path + 'hawaii.jpg',0)

# Show first one histogram
plt.subplot(121)
plt.title("Nevada plate")
plt.hist(None,None,[None,None]) 

# And the second one!
plt.subplot(122)
plt.title("Hawaii plate")
plt.hist(None,None,[None,None]) 
plt.show() 

Now, let's complete the method `histogram_moments()` that implements the retrieval of the central moments of the histogram shown above. This method takes as input:

- an image, and 
- the number of moments to be calculated

and returns an array containing those moments of the image's histogram.

In [ ]:
def histogram_moments(image,k):
    """ Compute central moments of the histogram of an image.   
    
        Args:
            image: input image
            k: number of moments to compute
                    
        Returns: 
            histogram_moments: array containing the histogram moments
    """   
    
    
    # Compute histogram
    hist = cv2.calcHist([image],[0],None,[256],[0,256]) # Keep the None in this function!

    # Compute mean average intensity/brightness of the image
    z_mean = np.dot(hist[hist.nonzero()[0]].flatten(),hist.nonzero()[0].flatten())/hist[hist.nonzero()[0]].sum()
    
    # Compute moments
    histogram_moments = np.zeros(k)
    for i in range(k):
        moment = 0.0
        for z in range(1,256):            
            moment += float(None-None)**None * float(None)        
            
        histogram_moments[i] = moment/hist.sum()
        
        # The previous code could be replaced by just one line!
        # histogram_moments[i] = np.average((np.arange(1,256) - z_mean)**i, weights = hist[1:256].flatten())
    
    return(histogram_moments)

You can use the next code to **test if the results are correct**:

In [ ]:
image = np.array([[10,60,20],[60,22,74],[72,132,2]], dtype=np.uint8)

moments = histogram_moments(image,6)

print(moments)

<font color='blue'>**Expected output: </font>**

    [1.00000000e+00 0.00000000e+00 1.50795062e+03 3.83609108e+04 6.08670794e+06 3.62329032e+08]

#### Invariance analysis

Now that we can obtain the first `k` moments of an image histogram, we are going to see if this method is invariant to scale and rotation. As in the UMA parking problem, our solution must be scale invariant, so let's check if it is.

### **<span style="color:green"><b><i>ASSIGNMENT 5: Checking the invariance of 1D moments</i></b></span>**

**What to do?** Check if **the first six 1D moments** of the histogram of an image, a rotated version of it, and a scaled version, are the same. Use [`np.array_equal()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array_equal.html) for that.

In [ ]:
# Assignment 5

# Read image and compute histogram moments
image = cv2.imread(images_path + 'hawaii.jpg',0)
moments = histogram_moments(None,6)

# Rotate image and compute histogram moments
image_rotated = np.rot90(None)
moments_rotated = histogram_moments(None,6)
    
# Resize image and compute histogram moments
image_scaled = cv2.resize(None, dsize=None, fx=None, fy=None) # keep the dsize=None
moments_scaled = histogram_moments(None,6)

# Compare results
print("Rotation invariance: ", np.array_equal(None,None))
print("Scale invariance: ", np.array_equal(None,None))

# Show the initial image
plt.subplot(131)
plt.title('Initial image')
plt.imshow(image, cmap='gray')

# Show the rotated version
plt.subplot(132)
plt.title('Rotated version')
plt.imshow(image_rotated, cmap='gray')

# Show the scaled version
plt.subplot(133)
plt.title('Scaled version')
plt.imshow(image_scaled, cmap='gray');

### <font color="blue"><b><i>Thinking about it (5)</i></b></font>

Now, **answer the following questions:**

- Is it invariant to rotation? If not, how can we turn this method into it?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- Is it invariant to scale? If not, how can we turn this method into it?

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>

### 6.2.3.2 Co-ocurrence matrix <a id="6232"></a>

Another technique also obtaining a statistical representation of the texture within a region is the **co-ocurrence matrix**, a square matrix $A(i,j)$ in which:

- $i$ and $j$ represent intensity values (e.g. 0 to 255). 
- The entry $a_{ij}$ indicates how many times the intensity $i$ co-occurs with intensity $j$ in some designated spatial relationships $P$ (texture pattern). 
- $P$ is given by a displacement vector $d = [dr, dc]$, where $dr$ and $dc$ are the displacement in rows and columns, respectively.

<center><img src="./images/co-ocu.png" width="650"/></center>

The issue with this approach is how to select the appropriate displacement $d$. Once the co-ocurrence matrix of a region has been computed, a number of features can be extracted from it:

- **Maximum probability:** gives us the strongest response to the texture pattern $P$$ \\[1pt]$

$\hspace{2cm} max_{ij}\ c_{ij}$

- **Energy:** minimum when all the entries $c_{ij}$ are identical (maximum uniformity)$ \\[1pt]$

$\hspace{2cm} \sum_{i=0}^{255}\sum_{j=0}^{255}\ c_{ij}^2$

- **Entropy:** measure randomness. Maximum value when all the entries $c_{ij}$ are identical (maximum entropy  $\rightarrow$ minimum energy)$ \\[1pt]$

$\hspace{2cm} -\sum_{i=0}^{255}\sum_{j=0}^{255}\ c_{ij} \ logc_{ij}$

- **Order k central moment**$ \\[1pt]$

$\hspace{2cm} \sum_{i=0}^{255}\sum_{j=0}^{255}\ (i-j)^k \ c_{ij}$

### **<span style="color:green"><b><i>ASSIGNMENT 6: Computing co-ocurrence matrices</i></b></span>**

Let's implement the method `co_ocurrence_matrix_features()`, which has to compute the normalized co-ocurrence matrix of `image` using the displacement vector `[dr,dc]` and normalizes it, obtaining `C(i,j)`. Note that `dr` and `dc` may take positive or negative values. Thereby, it takes as inputs:
- an image, 
- a 2-size displacement vector, and 
- a number of central moments to compute. 

and returns:
- a feature vector with size 3 + `n_moments` being: [`max_prob`, `energy`, `entropy`, `moments` (optional)]

In [ ]:
# Assignment 6

def co_ocurrence_matrix_features(image, d, n_moments):
    """ Compute features from a image using a co-ocurrence matrix.   
    
        Args:
            image: Binary image
            d: displacement vector
            n_moments: number of moment to be computed
                    
        Returns: 
            features: feature vector
    """   
    
    (n_r, n_c) = image.shape
    co = np.zeros((256,256))
    features = np.zeros(3+n_moments)
    
    # Compute image ranges to iterate from displacement vector
    
    if d[0] >= 0:
        range_rows = range(0, n_r-d[0], 1)
    else:
        range_rows = range(-d[0], n_r, 1)

    if d[1] >= 0:
        range_columns = range(0, n_c-d[1], 1)
    else:
        range_columns = range(-d[1], n_c, 1)

    # Compute co-ocurrence matrix    
    for r in range_rows:
        for c in range_columns:

            i = image[r,c]
            j = image[r+d[0],c+d[1]]

            co[i,j] += 1

    # Normalize co-ocurrence matrix
    co = co/np.sum(co)

    # Maximum probability
    features[0] = None

    # Energy
    features[1] = None

    # Entropy
    mask = np.where(co!=0, True, False)
    features[2] = None

    # Central moments
    for k in range(n_moments):
        moment = 0
        for i in range(co.shape[0]):
            for j in range(co.shape[1]):
                moment += ((None-None)**None)*co[None,None]
                
        features[3+k] = moment
        
    return np.round(features,5)

You can use the next code to **test if the results are right**:

In [ ]:
np.set_printoptions(suppress=True)

image = np.array([[10,60,20],[60,22,74],[72,132,2]], dtype=np.uint8)

features = co_ocurrence_matrix_features(image,d=[1,-2],n_moments=4)

print(features)

<font color='blue'>**Expected output: </font>**

    [0.5  0.5  0.693  1.  -19.  802.  -31996.  ]

### **<span style="color:green"><b><i>ASSIGNMENT 7: Studying the invariance of co-occurence matrices</i></b></span>**

Compare the results returned by `co_ocurrence_matrix_features()` when using the original image `hawaii.jpg`, with those returned by a rotated or scaled version of it.

In [ ]:
# Assignment 7

# Read image and compute co-occurence matrix features
image = cv2.imread(images_path + 'hawaii.jpg',0)
features = co_ocurrence_matrix_features(None,d=[1,-2],n_moments=4)

# Rotate image and compute co-occurence matrix features
image_rotated = np.rot90(None)
features_rotated = co_ocurrence_matrix_features(None,d=[1,-2],n_moments=4)
    
# Resize image and compute co-occurence matrix features
image = cv2.resize(None, dsize=None, fx=None, fy=None) # keep the dsize=None
features_scaled = co_ocurrence_matrix_features(None,d=[1,-2],n_moments=4)

# Compare results
print("Features original ", features,"\n Features rotated ", features_rotated, "\n Features scaled ", features_scaled)

### <font color="blue"><b><i>Thinking about it (6)</i></b></font>

Now, **answer the following question:**

- **Compare the invariance of each feature in the feature vector** and **comment why it is invariant or not to rotation and scale.**

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

## Conclusion

Awesome! This was a laborious and dense notebook, but you carried it through to the end!

In this notebook you have learned:

- how to compute non-central, central, scale-invariant and Hu moments for describing a region, and apply them to the plate number recognition problem.

- how to describe textures using 1D moments of the histogram and co-ocurrence matrices, using them in the context of the state identification problem.

## Extra

Usually, the co-ocurrence matrices **of the image rotated** 45, 90, and 135 degrees are also calculated. **What do you think this is due to?**  

**Implement this new procedure** for co-ocurrence matrices and then, check again the invariances. **What happened?**